__BIOBSS - Activity Indices__

_This notebook includes guidelines to help using BIOBSS for calculation of activity indices from ACC signal._

In [1]:
#Import BIOBSS and the other required packages

import biobss
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#help(biobss)

# Table of Contents
1.[ACC Sample Data](#sampledata)<br>
2.[ACC Signal Filtering](#acc_pre)<br>
3.[ACC Dataset Generation](#acc_dataset)<br>
4.[Activity indices from ACC signal](#act_ind)<br>
5.[ACC Feature Extraction](#acc_features)<br>

### __ACC Sample Data__
<a id="sampledata"></a>

In [2]:
#Load the sample data. The sample data file contains 100 10-s PPG segments. The samling rate of the segments is 64 Hz. 

data_dir=r'C:\Users\ipek.karakus\Desktop\biobss\sample_data'

filename='acc_sample_data.csv'
fs=32
L=60
data=pd.read_csv(os.path.join(data_dir,filename), header=None)
accx=np.asarray(data.iloc[2:fs*L+2,0]) #The first segment is selected to be used as sample signal. 
accy=np.asarray(data.iloc[2:fs*L+2,1]) #The first segment is selected to be used as sample signal. 
accz=np.asarray(data.iloc[2:fs*L+2,2]) #The first segment is selected to be used as sample signal. 


### __ACC Signal Filtering__
<a id="acc_pre"></a>

BIOBSS library provides a filtering function based on Scipy. This function can be used to implement Butterworth filter by defining the filter parameters. 

In [20]:
#or defining the parameters.

f_accx= biobss.preprocess.filter_signal(accx,fs,'lowpass',N=2,f_upper=10)
f_accy= biobss.preprocess.filter_signal(accy,fs,'lowpass',N=2,f_upper=10)
f_accz= biobss.preprocess.filter_signal(accz,fs,'lowpass',N=2,f_upper=10)

Additionally, pre-defined filter parameters can be used for each specific signal type. In order to use this option, signal_type should be selected as 'ACC'. 

In [21]:
#PPG signals can be filtered using the method below using pre-determined filter parameters

filtered_accx=biobss.preprocess.filter_signal(accx, fs, signal_type='ACC')
filtered_accy=biobss.preprocess.filter_signal(accy, fs, signal_type='ACC')
filtered_accz=biobss.preprocess.filter_signal(accz, fs, signal_type='ACC')

c:\Users\ipek.karakus\.conda\envs\biolib\lib\site-packages\biobss\imutools\acc_filter.py:11: UserWarning: Default parameters will be used for filtering. 2th order None lowpass filter with f2=10.
  warnings.warn(f"Default parameters will be used for filtering. {N}th order {method} {filter_type} filter with f2={f_upper}.")


### __ACC Dataset Generation__
<a id="acc_dataset"></a>

Due to the definitions of activity indices in the literature, some specific preprocessing steps should be applied on the raw signals. In other words, each activity index can be calculated using only specific datasets. BIOBSS package provides functions to automatically apply appropriate preprocessing steps (dataset generation) to prevent errors and make the process easier for the researchers.

In [22]:
ufxyz = biobss.imutools.generate_dataset(accx,accy,accz,fs,False,None,False,False,False),
ufm = biobss.imutools.generate_dataset(accx,accy,accz,fs,False,None,True,False,False),
ufm_modified = biobss.imutools.generate_dataset(accx,accy,accz,fs,False,None,True,False,True),
ufnm = biobss.imutools.generate_dataset(accx,accy,accz,fs,False,None,True,True,False),
fxyz = biobss.imutools.generate_dataset(accx,accy,accz,fs,True,'pre',False,False,False),
fxyz_modified = biobss.imutools.generate_dataset(accx,accy,accz,fs,True,'pre',False,False,True),
fmpre = biobss.imutools.generate_dataset(accx,accy,accz,fs,True,'pre',True,False,False),
specialxyz = biobss.imutools.generate_dataset(accx,accy,accz,fs,True,'pre',False,False,False,'highpass',4,0.2),
specialm = biobss.imutools.generate_dataset(accx,accy,accz,fs,True,'pre',True,False,False,'highpass',4,0.2),
fmpost = biobss.imutools.generate_dataset(accx,accy,accz,fs,True,'post',True,False,False),
fmpost_modified = biobss.imutools.generate_dataset(accx,accy,accz,fs,True,'post',True,False,True)
                    

### __Activity Indices from ACC Signals__
<a id="act_ind"></a>

In [32]:
pim = biobss.imutools.calc_activity_index(accx, accy, accz, signal_length=60, sampling_rate=fs, metric='PIM')
zcm = biobss.imutools.calc_activity_index(accx, accy, accz, signal_length=60, sampling_rate=fs, metric='ZCM')
tat = biobss.imutools.calc_activity_index(accx, accy, accz, signal_length=60, sampling_rate=fs, metric='TAT')
mad = biobss.imutools.calc_activity_index(accx, accy, accz, signal_length=60, sampling_rate=fs, metric='MAD')
enmo = biobss.imutools.calc_activity_index(accx, accy, accz, signal_length=60, sampling_rate=fs, metric='ENMO')
hfen = biobss.imutools.calc_activity_index(accx, accy, accz, signal_length=60, sampling_rate=fs, metric='HFEN')
ai = biobss.imutools.calc_activity_index(accx, accy, accz, signal_length=60, sampling_rate=fs, metric='AI', baseline_variance=[0.5,0.5,0.5])

c:\Users\ipek.karakus\.conda\envs\biolib\lib\site-packages\biobss\imutools\acc_activityindex.py:98: UserWarning: Threshold level is required for this activity index, but not provided. Standard deviation of the signal will be used as threshold.
  warnings.warn("Threshold level is required for this activity index, but not provided. Standard deviation of the signal will be used as threshold.")


### __ACC Feature Extraction__
<a id="acc_features"></a>

In [3]:
signals = {}
signals['accx'] =accx
signals['accy'] =accy
signals['accz'] =accz

In [6]:
biobss.imutools.get_acc_features(signals, sampling_rate=fs, magnitude=True)

{'accx_fft_mean': 0.15411387937890197,
 'accx_fft_std': 1.221193879724076,
 'accx_fft_mad': 0.1499930519823061,
 'accx_fft_min': 0.0019237381328550316,
 'accx_fft_max': 37.05885416666666,
 'accx_fft_range': 37.05693042853381,
 'accx_fft_median': 0.06990092258113458,
 'accx_fft_medad': 0.03154271172774663,
 'accx_fft_iqr': 0.07006396989971755,
 'accx_abmean': 435,
 'accx_npeaks': 158,
 'accx_skew': 1.1960001411169314,
 'accx_kurtosis': 3.9315288215960393,
 'accx_energy': 27147.09,
 'accx_f1sc': 2.1876925736474537,
 'accx_f2sc': 0.9148786645657963,
 'accx_f3sc': 0.19959846467090536,
 'accx_Entropy': -26.519738345521574,
 'accx_max_freq': 0.13333333333333333,
 'accy_fft_mean': 0.08629440048286408,
 'accy_fft_std': 0.16435688343753996,
 'accy_fft_mad': 0.08527381272127706,
 'accy_fft_min': 0.00046797536656737084,
 'accy_fft_max': 3.0859375,
 'accy_fft_range': 3.0854695246334325,
 'accy_fft_median': 0.0364463774423682,
 'accy_fft_medad': 0.01939112730216174,
 'accy_fft_iqr': 0.0497522631956